# 第9章 代码调优

## 9.1 典型的故事

案例背景：本书作者和Chris Van Wyk在讨论代码调优的问题，改进了C程序，将一个3000行的图形程序的运行时间减少了一半。

分析：
1. 监视程序的性能，确定每个函数需要花费的时间。几乎70%的运行时间都用在了内存分配函数malloc上。
2. 研究内存分配程序：插入计数代码，发现常见记录类型的空间分配次数是次常见记录类型的30倍。

优化：使用高速缓存原理，将最常见类型的空闲记录缓存在一个链表中，通过对链表的快速访问来处理常见的请求，总运行时间缩短为原先的45%。

## 9.2 急救方案锦集

**问题1：** 整数取模

```c
k = (j + rotdist) % n
```

C语言的模运算符%开销较大，可以使用算术运算代替模运算。

```c
k = j + rotdist;
if (k >= n):
    k -= n;
```

**问题2：** 函数、宏和内联代码

在C语言中，对某一函数进行内联编译，兼得了函数得简洁语义和宏得低廉开销。

**问题3：** 顺序搜索

对于现代计算机来说，将循环展开有助于避免管道阻塞、减少分支、增加指令级的并行性。

**问题4：** 计算球面距离

原始方案：将球面上5000个点组成的集合S表示成包含经度和纬度值的数组。对于序列中的每个点，计算它到S中每个点的距离，来确定S中和它最接近的点。

改进方案：使用$x$、$y$和$z$坐标表示球面上点的位置，使用一个数组，不仅包含每个点的经纬度，还包含了该点的三个笛卡儿坐标。

## 9.3 二分搜索

In [36]:
import random

random.seed(2023)

x = [random.randint(0, 10000) for _ in range(1000)]
x.sort()

In [37]:
x[:5]

[22, 38, 41, 42, 63]

**原始版本：**

In [38]:
def binarysearch(x, t):
    l = 0
    u = len(x) - 1
    while True:
        if l > u:
            p = -1
            break
        m = (l + u) // 2
        if x[m] < t:
            l = m + 1
        elif x[m] == t:
            p = m
            break
        else:
            u = m - 1
    return p

In [39]:
p = binarysearch(x, 63)
p, x[p]

(4, 63)

**第1个版本：**

改进方案：将不变式关系变为$x[l] < t \leqslant x[u]$和$l < u$。

In [40]:
def binarysearch1(x, t):
    n = len(x)
    l = -1
    u = n
    while l + 1 != u:
        m = (l + u) // 2
        if x[m] < t:
            l = m
        else:
            u = m
    p = u
    if p >= n or x[p] != t:
        p = -1
    return p

In [41]:
p = binarysearch1(x, 63)
p, x[p]

(4, 63)

**第2个版本：**

改进方案：利用$n=1000$这个条件，使用下限值1以及增量$i$表示，使得$l + i = u$

In [42]:
def binarysearch2(x, t):
    i = 512
    l = -1
    if x[511] < t:
        l = 1000 - 512
    while i != 1:
        nexti = i // 2
        if x[l + nexti] < t:
            l = l + nexti
            i = nexti
        else:
            i = nexti
    
    p = l + 1
    if p > 1000 or x[p] != t:
        p = -1
    return p

In [43]:
p = binarysearch2(x, 63)
p, x[p]

(4, 63)

**第3个版本：**

改进方案：加入智能编译器可能会执行的某些优化，简化第二个if语句，删除变量`nexti`，并从循环内的if语句中删除了对`nexti`的赋值。

In [44]:
def binarysearch3(x, t):
    i = 512
    l = -1
    if x[511] < t:
        l = 1000 - 512
    while i != 1:
        i = i // 2  
        if x[l + i] < t:
            l = l + i
            
    p = l + 1
    if p > 1000 or x[p] != t:
        p = -1
    return p

In [45]:
p = binarysearch3(x, 63)
p, x[p]

(4, 63)

**第4个版本：**

改进方案：展开了整个循环，消除了循环控制和i被2除的开销。

In [46]:
def binarysearch4(x, t):
    l = -1
    if x[511] < t:
        l = 1000 - 512
    if x[l + 256] < t:
        l += 256
    if x[l + 128] < t:
        l += 128
    if x[l + 64] < t:
        l += 64
    if x[l + 32] < t:
        l += 32
    if x[l + 16] < t:
        l += 16
    if x[l + 8] < t:
        l += 8
    if x[l + 4] < t:
        l += 4
    if x[l + 2] < t:
        l += 2
    if x[l + 1] < t:
        l += 1
    p = l + 1
    if p > 1000 or x[p] != t:
        p = -1
    return p

In [47]:
p = binarysearch4(x, 63)
p, x[p]

(4, 63)

In [48]:
import time

def test_time(algnum, n, numtests):
    x = [i for i in range(n)]
    start_time = time.time()
    
    for testnum in range(numtests):
        for i in range(n):
            match(algnum):
                case 1: 
                    assert(binarysearch(x, i) == i)
                case 2:
                    assert(binarysearch4(x, i) == i)
    clicks = (time.time() - start_time) * 1000
    print(algnum, n, numtests, int(clicks), "{:.1f}".format(int(clicks) * 1e6 / ( n * numtests)))

In [49]:
test_time(1, 1000, 10000)

1 1000 10000 8762 876.2


In [50]:
test_time(2, 1000, 10000)

2 1000 10000 5444 544.4


原始版本的运行时间为876.2纳秒，第4版循环展开后的程序运行时间为544.4纳秒，加速了37%。

## 9.4 原理

- 效率的角色：不成熟的优化是大量编程灾害的根源，会危机程序的正确性、功能性以及可维护性。
- 度量工具：当效率很重要时，第一步就是对系统进行性能监视，多数的时间都消耗在少量的热点代码上。
- 设计层面：只有在确信没有更好的解决方案时，才考虑进行代码调优。
- 双刃剑：使用if语句替换模运算有时可以使速度加倍，有时对运行时间没什么影响。